<center>
<img src="https://2223moodle.isel.pt/pluginfile.php/1/theme_adaptable/logo/1631635830/logo-isel_inv3.png" width="250">
<h3>Licenciatura em Engenharia Informática e Multimédia</h3><br>
<br>
<h2>Processamento de Imagem e Visão (PIV)</h3>
<h3>2º Trabalho Laboratorial – Estimação e classificação de movimento </h3>
<br>

<br>
Trabalho Realizado por:<br>
Gonçalo Silva <b>A48328</b><br>
Diogo Lobo <b>A48168</b><br>
Turma 52D<br><br>
Docente: João Pedro Costa <br>
<br>
27 de Novembro 2024
</center>


In [1]:
# Imports
import cv2
import numpy as np
import time
import pygame
from pygame.locals import QUIT
from pygame import surfarray

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Inicialização dos tamanhos da janela e objetos
window_width = 1280
window_height = 700
cube_size = 50
cube_x = window_width // 4 - cube_size
cube_y = window_height // 2 - cube_size
cube_color = (0, 0, 255)

In [3]:
#Inicialização da Camera 
camera = 0
cam = cv2.VideoCapture(camera)

running = True


pygame.init()

window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Webcam in Pygame")

running = True

if cam.isOpened():
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        flag, Bframe = cam.read()
        if flag:
            # Converte de BGR (OpenCV) para RGB (Pygame)
            Bframe = cv2.cvtColor(Bframe, cv2.COLOR_BGR2RGB)
            Bframe = np.rot90(Bframe)  # Pygame espera a imagem transposta
            pygame_surface = pygame.surfarray.make_surface(Bframe)
            window.blit(pygame_surface, (window_width // 2, window_height // 7))  # Centraliza na tela

        pygame.display.update()  # Atualiza a tela

    # Libera recursos ao sair
    cam.release()
    pygame.quit()

else:
    print("Error: Could not open Camera")


In [4]:
import cv2
import numpy as np
import time

camera = 0
cam = cv2.VideoCapture(camera)

thr = 10
kernelSize = 9
previousFrame = None
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernelSize,kernelSize),(-1,-1))

while True:
    flag, img = cam.read()
    if flag:
        imgE = cv2.medianBlur(img,kernelSize)
        if previousFrame is None:
            previousFrame = img
        diff_image = cv2.absdiff(img,previousFrame)
        _ ,bw1 = cv2.threshold(diff_image,thr,255,cv2.THRESH_BINARY)
        _, bw2 = cv2.threshold(np.uint8(bw1.sum(axis=2)),1,255,cv2.THRESH_BINARY)
        erode_image = cv2.erode(bw2,kernel)
        dilate_image = cv2.dilate(erode_image,kernel,iterations=2)
        (num_labels,label_image,stats,centroids) = cv2.connectedComponentsWithStats(dilate_image)
        previousFrame = img.copy()
        motionFlag = False
        for i in range(1,num_labels):
            if(stats[i,4]>0.05*bw2.shape[0]*bw2.shape[1]):
                cv2.rectangle(img,(stats[i,0],stats[i,1]),(stats[i,0]+stats[i,2],stats[i,1]+stats[i,3]),(0,0,255),thickness=2)
                motionFlag = True
        if motionFlag:
            cv2.putText(img, 'Motion Detected', (0,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
        
        cv2.imshow('Video in',img)

    key = cv2.waitKey(100)
    if key==27:
        cv2.destroyAllWindows()
        cam.release()
        break

    